In [1]:
"""
Created November 2023

@author: Annika Deutsch
@date: 11/2023
@title: roc_curves.ipynb
@description: For a subset of pulsars with known binary companions, calculate roc curves to determine the ideal search radius 
to be performing the query in
"""

'\nCreated November 2023\n\n@author: Annika Deutsch\n@date: 11/2023\n@title: roc_curves.ipynb\n@description: For a subset of pulsars with known binary companions, calculate roc curves to determine the ideal search radius \nto be performing the query in\n'

## Imports

In [2]:
import astropy
import astropy.units as u
import astroquery
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import quantity_support
from astropy.time import Time 
import os
from astropy.io.votable import parse_single_table
from astropy.time import Time
import pytest
from astropy.table import Table, vstack
import csv
import pandas as pd
from astropy.io.votable import from_table, writeto

## Load in Table of Matches

In [3]:
npz = np.load('/home/billee/Binary-Pulsar-Distances/Binary_Pulsar_Distances/cross-matches/matches_10arcsec_11-1_sourceseps.npz', allow_pickle=True)
table = Table(npz['arr_0'])

In [29]:
cond = table['Companion Pulsar'] == 'J1023+0038'
table[cond]

Companion Pulsar,Pulsar RA,Pulsar DEC,Binary,Binary Companion,solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,source_seps,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,dist
str12,str16,str16,str6,str16,int64,object,int64,int64,float64,float64,float32,float64,float32,float64,float64,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,int16,bool,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32,int16,int16,int16,float32,float32,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,float32,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,int16,float32,float32,int16,float32,object,float64,float64,float64,float64,bool,bool,int16,bool,bool,bool,bool,bool,bool,bool,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,float64
J1023+0038,10:2

## Load in Tables of "Known" Matches (sets with which to determine true vs/ false)

In [18]:
# jennings paper
jennings_psrs = ['J0337+1715', 'J0437-4715', 'J1012+5307', 'J1023+0038', 'J1227-4853', 'J1302-6350', 'J1417-4402', 'J1431-4715', 
                   'J1723-2837', 'J2032+4127', 'J2129-0429', 'J2339-0533', 'J0045-7319', 'J0348+0432', 'J1024-0719', 'J1048+2339',
                   'J1311-3430', 'J1628-3205', 'J1810+1744', 'J1816+4510', 'J1957+2516', 'J2215+5135']
jennings_gmags = [18.08, 20.41, 19.63, 16.27, 18.08, 9.63, 15.79, 17.75, 15.55, 11.36, 16.84, 18.97, 16.22, 20.64, 19.18, 19.65, 20.53,
                  19.52, 20.08, 18.22, 20.30, 19.24]

jennings = Table([jennings_psrs, jennings_gmags], names=('Companion Pulsar', 'G mag'))

## Create tables of radii, prob detec., prob fa, and save in astropy table

In [35]:
radii = np.linspace(0.1,10,100)
p_d = np.zeros(len(radii))
p_fa = np.zeros(len(radii))

roc = Table([radii, p_d, p_fa], names=('Search Radii', 'Probability of Detection', 'Probaility of False Alarm'))

## Calculate Probabilities

In [ ]:
cond = Angle(table['source_seps'], unit=u.deg).arcsec <= 0.1
temp_table = table[cond]
temp_table

In [ ]:
# iterate through radii and calculate probabilities
for i in range(len(radii)):
    cond = Angle(table['source_seps'], unit=u.deg).arcsec <= radii[i] # limit table to this radius
    temp_table = table[cond]
    npsrs = len(temp_table)

    # iterate through each pulsar in temp_table, and check if it is in reference list
    ntrue = 0
    nfalse = 0
    for j in temp_table:
        hit = False
        for k in jennings:
            if temp_table[j]['Companion Pulsar'] == jennings[k]['Companion Pulsar']:
                hit = True
        if hit:
            ntrue += 1
        else:
            nfalse += 1
    
    p_d[i] = ntrue / npsrs
    p_fa[i] = nfalse / npsrs

ZeroDivisionError: division by zero

In [11]:
# calculate probability of detection (pd) and probability of false alarm (pfa)

total_psrs = 1
n_hits_true = 1
n_hits_false = 1

pd = n_hits_true / total_psrs
pfa = n_hits_false / total_psrs

#calculate the number of total individual pulsars (no reduplication)
current = 'J0024-7204E'
psrs_with_hits = [current] # total number of PULSARS with hits, distinct from total number of hits
pwh_binary = ['DD']
for b in range(len(subset['Pulsar'])):
    if subset['Pulsar'][b] != current:
        psrs_with_hits.append(subset['Pulsar'][b])
        pwh_binary.append(subset['Binary'][b])
        current = subset['Pulsar'][b] 

In [12]:
# toy model with a small enough separation that it is easy to go in by hand to do:
subset[400:415]

Pulsar,Pulsar RA,Pulsar DEC,ra,dec,Binary,Separation
,,,,,,deg
str12,str16,str16,str18,str19,str6,float64
J0024-7204Y,00:24:01.4026,-72:04:41.8363,6.001653822714394,-72.08071400891852,ELL1,0.0027474776346458223
J0337+1715,03:37:43.82589,+17:15:14.828,54.43261340914772,17.254115461920154,ELL1,6.29892323013655e-06
J0348+0432,03:48:43.639000,+04:32:11.4580,57.18183331779655,4.5365161957787254,ELL1,4.1389910062538866e-06
J0437-4715,04:37:15.8961737,-47:15:09.110714,69.31662758499185,-47.25268764316094,T2,0.00030978067618349044
J0514-4002A,05:14:06.69271,-40:02:48.8930,78.52799552909461,-40.04700266794965,DDH,0.00012135601305043101
J0514-4002A,05:14:06.69271,-40:02:48.8930,78.5280939738137,-40.046497857750886,DDH,0.00044615264121269373
J0514-4002A,05:14:06.69271,-40:02:48.8930,78.52753203005061,-40.04733050061802,DDH,0.0004964042623038699
J0514-4002A,05:14:06.69271,-40:02:48.8930,78.52797467644648,-40.0464105346105,DDH,0.0005087072581887858
